In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl.drawing.image import Image
from io import BytesIO
from numba import njit

In [2]:
# Evaluation functions
@njit
def f1(pop):
    return np.sum(pop ** 2, axis=1)

@njit
def f2(pop):
    return np.sum(100 * (pop[:, 1:] - pop[:, :-1] ** 2) ** 2 + (1 - pop[:, :-1]) ** 2, axis=1)

In [3]:
def initialize_population(pop_size, dimensions, domain, binary=False):
    if binary:
        return np.random.randint(2, size=(pop_size, dimensions * 16))  # Binary
    return np.random.uniform(domain[0], domain[1], (pop_size, dimensions))

In [4]:
def tournament_selection(pop, scores, k=3):
    selected = np.random.choice(range(len(pop)), k)
    best = selected[np.argmin([scores[i] for i in selected])]
    return pop[best]

In [5]:
def crossover(parent1, parent2, binary=False):
    if binary:
        point = np.random.randint(1, len(parent1) - 1)
        child1 = np.concatenate((parent1[:point], parent2[point:]))
        child2 = np.concatenate((parent2[:point], parent1[point:]))
    else:
        alpha = np.random.rand()
        child1 = alpha * parent1 + (1 - alpha) * parent2
        child2 = alpha * parent2 + (1 - alpha) * parent1
    return child1, child2

In [6]:
def mutate(individual, mutation_rate=0.01, binary=False, domain=(-5, 5)):
    if binary:
        for i in range(len(individual)):
            if np.random.rand() < mutation_rate:
                individual[i] = 1 - individual[i]
    else:
        individual += np.random.normal(0, 0.1, size=individual.shape)
        individual = np.clip(individual, domain[0], domain[1])
    return individual

In [7]:
def evolutionary_algorithm(pop_size, dimensions, domain, generations, eval_func, binary=False):
    pop = initialize_population(pop_size, dimensions, domain, binary)
    best_scores = []
    for _ in range(generations):
        scores = eval_func(pop)
        best_scores.append(np.min(scores))

        new_pop = []
        for _ in range(pop_size // 2):
            parent1 = tournament_selection(pop, scores)
            parent2 = tournament_selection(pop, scores)
            child1, child2 = crossover(parent1, parent2, binary)
            new_pop.append(mutate(child1, binary=binary, domain=domain))
            new_pop.append(mutate(child2, binary=binary, domain=domain))
        pop = np.array(new_pop)

    return best_scores

In [8]:
# Experiment configuration
pop_size = 50
dimensions = 10
generations = 200
binary = False
domain_f1 = (-5.12, 5.12)
domain_f2 = (-5, 10)

In [9]:
def run_experiments():
    results = {}
    # For F1 and real-valued representation
    results['F1_real'] = [evolutionary_algorithm(pop_size, dimensions, domain_f1, generations, f1, binary=False) for _
                          in range(100)]
    # For F2 and real-valued representation
    results['F2_real'] = [evolutionary_algorithm(pop_size, dimensions, domain_f2, generations, f2, binary=False) for _
                          in range(100)]
    # For F1 and binary representation
    results['F1_binary'] = [evolutionary_algorithm(pop_size, dimensions, domain_f1, generations, f1, binary=True) for _
                            in range(100)]
    # For F2 and binary representation
    results['F2_binary'] = [evolutionary_algorithm(pop_size, dimensions, domain_f2, generations, f2, binary=True) for _
                            in range(100)]
    return results

In [10]:
def save_results_to_excel(results):
    with pd.ExcelWriter("results.csv", engine="openpyxl") as writer:
        for name, data in results.items():
            # Convert results to a NumPy array
            data = np.array(data)

            # Raw data - save to sheet
            df_raw = pd.DataFrame(data)
            df_raw.to_excel(writer, sheet_name=f'{name}_raw', index=False)

            # Mean of results - add chart
            mean_data = np.mean(data, axis=0)
            df_mean = pd.DataFrame(mean_data, columns=['Mean'])
            df_mean.to_excel(writer, sheet_name=f'{name}_mean', index=False)

            # Generate and save chart as image
            plt.figure()
            plt.plot(mean_data, label=f'Average Best Score ({name})')
            plt.xlabel("Pokolenie")
            plt.ylabel("Wartość funkcji celu")
            plt.legend()
            plt.title(f'Średnia wartość najlepszych wyników - {name}')

            # Save chart to memory
            img_data = BytesIO()
            plt.savefig(img_data, format='png')
            img_data.seek(0)
            plt.close()

            # Add chart to worksheet
            workbook = writer.book
            worksheet = workbook[f'{name}_mean']
            img = Image(img_data)
            worksheet.add_image(img, 'E5')

In [11]:
# Running experiments and saving results
results = run_experiments()
save_results_to_excel(results)